**Цель планируемого А/В-теста:** Разобраться, влияет ли то, кто из соперников подавал, на результаты Надаля. 
Если да, как именно

In [1]:
import numpy as np
import pandas as pd

In [2]:
from scipy.stats import norm, chi2_contingency
import statsmodels.api as sm

Результаты Надаля на глиняных и других кортах — геймы, в которых он подавал:

In [3]:
contingency_table = pd.DataFrame([[1090, 171], [2747, 446]],
                                 index=['Clay', 'Nonclay'],
                                 columns=['WIN', 'LOSS'])
contingency_table.loc['Total'] = contingency_table.sum(axis=0)
contingency_table['Total'] = contingency_table.sum(axis=1)
contingency_table

,WIN,LOSS,Total
Clay,1090,171,1261
Nonclay,2747,446,3193
Total,3837,617,4454


In [4]:
#                     RESULT
#                 WIN  LOSS TOTAL
#         Clay    1090 171  1261
# Surface Nonclay 2747 446  3193
#         Total   3837 617  4454

Результаты Надаля на глиняных и других кортах — геймы, в которых он принимал подачу:

In [5]:
contingency_table_1 = pd.DataFrame([[570, 692], [911, 2269]],
                                    index=['Clay', 'Nonclay'],
                                    columns=['WIN', 'LOSS'])
contingency_table_1.loc['Total'] = contingency_table_1.sum(axis=0)
contingency_table_1['Total'] = contingency_table_1.sum(axis=1)
contingency_table_1

,WIN,LOSS,Total
Clay,570,692,1262
Nonclay,911,2269,3180
Total,1481,2961,4442


In [6]:
#                     RESULT
#                 WIN  LOSS TOTAL
#         Clay    570  692  1262
# Surface Nonclay 911  2269 3180
#         Total   1481 2961 4442

Если мы не будем принимать во внимание вид покрытия, то сможем получить таблицу сопряженности, отражающую частоту наших переменных, со следующим наполнением:

In [7]:
contingency_table_3 = pd.DataFrame(index=['подавал', 'принимал', 'Total'],
                                   columns=['WIN', 'LOSS', 'Total'])

contingency_table_3.loc['подавал'] = contingency_table.loc['Total']
contingency_table_3.loc['принимал'] = contingency_table_1.loc['Total']
contingency_table_3.loc['Total'] = contingency_table_3.sum()
contingency_table_3

,WIN,LOSS,Total
подавал,3837,617,4454
принимал,1481,2961,4442
Total,5318,3578,8896


In [8]:
#              RESULT
#           WIN  LOSS TOTAL
# подавал   3837 617  4454
# принимал  1481 2961 4442
# Total     5318 3578 8896

Н0: p1 = p2 </br>
H1: p1 != p2

H0: Нет статистически значимой разницы в долях выигранных и проигранных геймов Надаля в зависимости от того, кто из соперников подает.</br>
H1: Существует статистически значимая разница в долях выигранных и проигранных геймов Надаля в зависимости от того, кто из соперников подает.

In [9]:
#   Версия 1. Прямые вычисления

s1 = 3837            #  число успехов  выборка А
n1 = 617 + s1      #  число испытаний   выборка А
s2 = 1481             #  число успехов     выборка Б
n2 = 2961  + s2     #  число испытаний   выборка Б
p1 = s1/n1               #  доля побед в выборке А
p2 = s2/n2               #  доля побед в выборке Б
p = (s1 + s2)/(n1+n2)    #  доля побед для объединенных выборок A и Б
z = (p2-p1)/ ((p*(1-p)*((1/n1)+(1/n2)))**0.5)     #  z-метка

p_value = norm.cdf(-abs(z)) # функция рапсределения стандартного нормального рапсределения в точке

#  z-метка и p-значение
result = ['{:.12f}'.format(a) for a in (abs(z), p_value * 2)]
print('Значение критерия статистики:', result[0])
print('p-value:', result[1])

Значение критерия статистики: 50.787212211865
p-value: 0.000000000000


p-value < 0.05, следовательно, нулевая гипотеза о равенстве вероятности выигрыша отвергается.

Значение z-метки - результат разницы между оценками вероятностей успеха для двух выборок (подача соперников и подача Надаля) с учетом их размеров и общей оценки вероятности успеха. </br>

В контексте z-теста для различия долей, большая z-метка указывает на значительное стандартное отклонение между двумя группами, что подразумевает сильные статистические различия в долях успеха. </br>

В данном случае, высокая z-метка может быть обусловлена существенными различиями между долями выигранных и проигранных геймов при подаче соперников и подаче Надаля. </br>

Чем больше различие между двумя долями (p1 и p2), тем больше будет значение z-метки. Значение z-метки и p-значения позволяют вам сделать вывод о том, насколько статистически значимы эти различия. В данном случае, так как p-значение близко к нулю, мы отвергаем нулевую гипотезу и считаем различия статистически значимыми. </br>

In [10]:
#   Версия 2. Используем  proportions_ztest

z1, p_value1 = sm.stats.proportions_ztest([s1, s2],[n1, n2])

#  z-метка и p-значение
print(['{:.12f}'.format(b) for b in (z1, p_value1)])

['50.787212211865', '0.000000000000']
